# Asian Options Pricing & Hedging Tool
Course: FE 620: Pricing & Hedging | Stevens Institute of Technology

Advisor: Dan Pirjol

Group: Theo Dimitrasopoulos, Will Kraemer, Vaikunth Seshadri, Snehal Rajguru

Link: https://colab.research.google.com/drive/1cImw6GuMlUYoXtHtEnyuP7lXsAlko2r6

*Version: v10.0*

## **Runtime comments:**

* Click the execution button on the top left corner of each code cell to execute it. To run all cells in descending order, go to the menu bar at the top and click Runtime -> Run all (or use the **Ctrl-F9** or **⌘-F9** hotkey for Windows and MacOSX respectively);
* If the code is running slowly, go to Runtime -> Change runtime type, and change the Runtime Shape to High-RAM from the dropdown menu;
* The **"!pip install"** lines under the Python packages section (i.e. lines 3-5) only need to be executed the first time you run the notebook. If you receive the message **"Requirement already satisfied:"**, wrap them in treble quotes (add the quotes in lines 2 & 6)*.

## Python Packages

In [1]:
# Install packages
'''
!pip install numpy
!pip install matplotlib
!pip install scipy
'''

# Import Packages
import random
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

## Inputs

### Underlying Asset Variables
Insert the desired values in the form fields (right) or directly to the code (left):

In [2]:
#@title Pricer Parameters:

# Initial Underlying Price
S0 = 100#@param {type:"number"}

# Risk-free rate (also known as the drift coefficient)
r = 0.15#@param {type:"number"}

# Dividend Yield Rate
q = 0.0#@param {type:"number"}

# Maturity
T = 1.0#@param {type:"number"}

# Strike
K = 95#@param {type:"number"}

# Volatility (also known as the diffusion coefficient)
sigma = 0.3#@param {type:"number"}

# Number of Iterations for Monte Carlo Simulation
it = 100000#@param {type:"integer"}

# Time Steps
N = 10000#@param {type:"integer"}

## Plotting Parameters

### Aesthetic Parameters

In [3]:
#@title Element Sizes:

# Universal Plot width
width = 25 #@param {type:"integer"}

# Universal Plot height
height =  14#@param {type:"integer"}

# Universal xtick size
xtick_size = 8 #@param {type:"integer"}

# Universal ytick size
ytick_size =  8#@param {type:"integer"}

# Universal title font size
title_size = 18 #@param {type:"integer"}

# Universal xlabel font size
xlabel_size = 12 #@param {type:"integer"}

# Universal xlabel font size
ylabel_size = 12 #@param {type:"integer"}

# Universal legend font size
legend_size = 10 #@param {type:"integer"}

### Execute Parameters

In [4]:
#Set parameters
plt.rcParams['figure.figsize'] = (width,height)
params = {'text.color' : 'black',
          'xtick.color' : 'w',
          'ytick.color' : 'w',
          'xtick.labelsize' : xtick_size,
          'ytick.labelsize' : ytick_size,
          'legend.loc' : 'upper left',
         }
plt.rcParams.update(params)

## Definitions

### Brownian Path Generator

In [5]:
def bm_paths(N):
  rand = random.random()
  dt = 1. / N
  b_dt = np.random.normal(0., 1., N)*np.sqrt(dt)
  W = np.cumsum(b_dt)
  return W

### Geometric Brownian Path Generator

In [6]:
def gbm_paths(r,q,N,T,S0,sigma):
  t = np.linspace(0.,1.,N)
  dt = T/N
  bt = np.random.normal(0., 1., int(N)) * np.sqrt(dt)
  W = np.cumsum(bt)
  S_i = S0 * np.exp(sigma * W + (r - q - 0.5 * (sigma**2)) * t)
  return S_i

### Black-Scholes Theoretical Price

In [7]:
def bs_call(r,q,T,K,S0,sigma):
  G0 = S0 * np.exp(0.5 * (r - q) * T - ((sigma**2) * T)/12)
  Sigma_G = sigma/np.sqrt(3)
  d1 = (1/(Sigma_G * np.sqrt(T))) * (np.log(G0/K) + 0.5 * (Sigma_G**2) * T)
  d2 = (1/(Sigma_G * np.sqrt(T))) * (np.log(G0/K) - 0.5 * (Sigma_G**2) * T)
  c = np.exp(-r * T) * (G0 * norm.cdf(d1) - K * norm.cdf(d2))
  return c

def bs_put(r,q,T,K,S0,sigma):
  G0 = S0 * np.exp(0.5 * (r - q) * T - ((sigma**2) * T)/12)
  Sigma_G = sigma/np.sqrt(3)
  d1 = (1/(Sigma_G * np.sqrt(T))) * (np.log(G0/K) + 0.5 * (Sigma_G**2) * T)
  d2 = (1/(Sigma_G * np.sqrt(T))) * (np.log(G0/K) - 0.5 * (Sigma_G**2) * T)
  p = np.exp(-r * T) * (K * norm.cdf(-d2) - G0 * norm.cdf(-d1))
  return p

### Monte Carlo Simulator with Arithmetic Average

In [8]:
# Call Options:
def mc_call_arithm(it,r,q,T,K,N,S0,sigma):
  mc_call_arithm_payoffs = []
  for i in range(1,it):
    S = gbm_paths(r,q,N,T,S0,sigma)
    S_arithm = np.mean(S)
    mc_call_arithm_payoffs.append(max(S_arithm - K, 0))
  c = np.exp(-r * T)*np.mean(mc_call_arithm_payoffs)
  return c

# Put Options:
def mc_put_arithm(it,r,q,T,K,N,S0,sigma):
  mc_put_arithm_payoffs = []
  for i in range(1,it):
    S = gbm_paths(r,q,N,T,S0,sigma)
    S_arithm = np.mean(S)
    mc_put_arithm_payoffs.append(max(K - S_arithm, 0))
  p = np.exp(-r * T)*np.mean(mc_put_arithm_payoffs)
  return p

### Monte Carlo Simulator with Geometric Average

In [9]:
# Call Options:
def mc_call_geom(it,r,q,T,K,N,S0,sigma):
  mc_call_geom_payoffs = []
  for i in range(1,it):
    S = gbm_paths(r,q,N,T,S0,sigma)
    S_geom = np.exp(np.mean(np.log(S)))
    mc_call_geom_payoffs.append(max(S_geom - K, 0))
  c = np.exp(-r * T)*np.mean(mc_call_geom_payoffs)
  return c

# Put Options:
def mc_put_geom(it,r,q,T,K,N,S0,sigma):
  mc_put_geom_payoffs = []
  for i in range(1,it):
    S = gbm_paths(r,q,N,T,S0,sigma)
    S_geom = np.exp(np.mean(np.log(S)))
    mc_put_geom_payoffs.append(max(K - S_geom, 0))
  p = np.exp(-r * T)*np.mean(mc_put_geom_payoffs)
  return p

## Performance Tests

### Black-Scholes vs. Monte Carlo: Call Options with Geometric Average

In [10]:
print('- - - - - - - - - - - - - - - - -')

for i in range(95,110,5):
  bs = bs_call(r,q,T,i,S0,sigma)
  mc = mc_call_geom(it,r,q,T,i,N,S0,sigma)
  error = np.abs((bs-mc)/bs)*100
  print('G_c(K=%d,T=%d):'%(i,T))
  print('Black-Scholes:',bs)
  print('Monte Carlo:', mc)
  print('Error: %f%%' % error)
  print('- - - - - - - - - - - - - - - - -')

- - - - - - - - - - - - - - - - -
G_c(K=95,T=1):
Black-Scholes: 12.508538481017892
Monte Carlo: 12.49780985501723
Error: 0.085770%
- - - - - - - - - - - - - - - - -
G_c(K=100,T=1):
Black-Scholes: 9.61215966961383
Monte Carlo: 9.579155249180117
Error: 0.343361%
- - - - - - - - - - - - - - - - -
G_c(K=105,T=1):
Black-Scholes: 7.185865725921304
Monte Carlo: 7.203453980969517
Error: 0.244762%
- - - - - - - - - - - - - - - - -


### Black-Scholes vs. Monte Carlo: Put Options with Geometric Average

In [11]:
print('- - - - - - - - - - - - - - - - -')

for i in range(95,110,5):
  bs = bs_put(r,q,T,i,S0,sigma)
  mc = mc_put_geom(it,r,q,T,i,N,S0,sigma)
  error = np.abs((bs-mc)/bs)*100
  print('G_p(K=%d,T=%d):'%(i,T))
  print('Black-Scholes:',repr(bs))
  print('Monte Carlo:',repr(mc))
  print('Error:',repr(error),'%')
  print('- - - - - - - - - - - - - - - - -')

- - - - - - - - - - - - - - - - -
G_p(K=95,T=1):
Black-Scholes: 2.194652455717922
Monte Carlo: 2.1742576087734857
Error: 0.9292973423331738 %
- - - - - - - - - - - - - - - - -
G_p(K=100,T=1):
Black-Scholes: 3.6018135264391438
Monte Carlo: 3.6163372763434825
Error: 0.40323436506990096 %
- - - - - - - - - - - - - - - - -
G_p(K=105,T=1):
Black-Scholes: 5.479059464871914
Monte Carlo: 5.509022386772284
Error: 0.546862506101145 %
- - - - - - - - - - - - - - - - -


### Asian call option test cases (Linetsky 2002):


#### **Test Cases: Parameters**
**%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%**

**C &emsp; r &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; σ &emsp; &emsp; &emsp; &emsp;T &emsp; S0&emsp; &emsp; EE&emsp; &emsp; &emsp; &emsp; &emsp; &emsp; MC &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; %Err**

1 &emsp; 0.0200 &emsp; 0.10 &emsp; 1 &emsp; 2.0 &emsp; 0.05602 &emsp; 0.0559860415 &emsp; 0.017

2 &emsp; 0.1800 &emsp; 0.30 &emsp; 1 &emsp; 2.0 &emsp; 0.21850 &emsp; 0.2183875466 &emsp; 0.059

3 &emsp; 0.0125 &emsp; 0.25 &emsp; 2 &emsp; 2.0 &emsp; 0.17250 &emsp; 0.1722687410 &emsp; 0.063

4 &emsp; 0.0500 &emsp; 0.50 &emsp; 1 &emsp; 1.9 &emsp; 0.19330 &emsp; 0.1931737903 &emsp; 0.084

5 &emsp; 0.0500 &emsp; 0.50 &emsp; 1 &emsp; 2.0 &emsp; 0.24650 &emsp; 0.2464156905 &emsp; 0.095

6 &emsp; 0.0500 &emsp; 0.50 &emsp; 1 &emsp; 2.1 &emsp; 0.30640 &emsp; 0.3062203648 &emsp; 0.106

7 &emsp; 0.0500 &emsp; 0.50 &emsp; 2 &emsp; 2.0 &emsp; 0.35030 &emsp; 0.3500952190 &emsp; 0.146

**%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%**

**Notes:**
* *(EE = Eigenfunction Expansion i.e. the Black-Scholes analytic result in
this algorithm; MC = Monte-Carlo estimate);*
* *All test cases have a strike K = 2.0 and a dividend yield q = 0.0;*

#### Set K = 2.0, q = 0.0:

In [12]:
# In all test cases, K = 2.0 and q = 0.0:
K = 2.0
q = 0.0

#### Test Cases:

In [13]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 1:
r = 0.02
sigma = 0.1
T = 1
S0 = 2.0

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,0.02,q,1,2.0,N,2.0,0.1)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 1 (r=0.02, sigma=0.10, T=1, S0=2.0):')
print('G_c(K=%d,T=%d):'%(2.0,1))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')

- - - - - - - - - - - - - - - - -


IndexError: invalid index to scalar variable.

In [ ]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 2:
r = 0.18
sigma = 0.30
T = 1.0
S0 = 2.0

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,r,q,T,K,N,S0,sigma)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 2 (r = 0.18, sigma = 0.30, T = 1, S0 = 2.0):')
print('G_c(K=%d,T=%d):'%(2.0,1))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')

In [ ]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 3:
r = 0.0125
sigma = 0.25
T = 2.0
S0 = 2.0

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,r,q,T,K,N,S0,sigma)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 3 (r = 0.0125, sigma = 0.25, T = 2, S0 = 2.0):')
print('G_c(K=%d,T=%d):'%(2.0,2))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')

In [ ]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 4:
r = 0.05
sigma = 0.50
T = 1.0
S0 = 1.9

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,r,q,T,K,N,S0,sigma)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 4 (r = 0.05, sigma = 0.50, T = 1, S0 = 1.9):')
print('G_c(K=%d,T=%d):'%(2.0,1))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')

In [ ]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 5:
r = 0.05
sigma = 0.50
T = 1.0
S0 = 2.0

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,r,q,T,K,N,S0,sigma)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 5 (r = 0.05, sigma = 0.50, T = 1, S0 = 2.0):')
print('G_c(K=%d,T=%d):'%(2.0,1))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')

In [ ]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 6:
r = 0.05
sigma = 0.50
T = 1.0
S0 = 2.1

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,r,q,T,K,N,S0,sigma)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 6 (r = 0.05, sigma = 0.50, T = 1, S0 = 2.1):')
print('G_c(K=%d,T=%d):'%(2.0,1))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')

In [ ]:
print('- - - - - - - - - - - - - - - - -')
# Test Case 7:
r = 0.05
sigma = 0.50
T = 2.0
S0 = 2.0

bs = bs_call(r,q,T,K,S0,sigma)
mc = mc_call_arithm(it,r,q,T,K,N,S0,sigma)[0]
error = np.abs((bs-mc)/bs)*100
print('Case 7 (r = 0.05, sigma = 0.50, T = 2.0, S0 = 2.0):')
print('G_c(K=%d,T=%d):'%(2.0,2))
print('Black-Scholes:',repr(bs))
print('Monte Carlo:',repr(mc))
print('Error:',repr(error),'%')

print('- - - - - - - - - - - - - - - - -')